# TMDB 5000 Movie Dataset으로 장르기반 추천시스템 만들기

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import koreanize_matplotlib

In [2]:
data = pd.read_csv("https://raw.githubusercontent.com/haram4th/ablearn/main/tmdb_5000_movies.csv")
data

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-07-16,1084939099,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-03-07,284139100,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4798,220000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",NaN,9367,"[{""id"": 5616, ""name"": ""united states\u2013mexi...",es,El Mariachi,El Mariachi just wants to play his guitar and ...,14.269792,"[{""name"": ""Columbia Pictures"", ""id"": 5}]","[{""iso_3166_1"": ""MX"", ""name"": ""Mexico""}, {""iso...",1992-09-04,2040920,81.0,"[{""iso_639_1"": ""es"", ""name"": ""Espa\u00f1ol""}]",Released,"He didn't come looking for trouble, but troubl...",El Mariachi,6.6,238
4799,9000,"[{""id"": 35, ""name"": ""Comedy""}, {""id"": 10749, ""...",NaN,72766,[],en,Newlyweds,A newlywed couple's honeymoon is upended by th...,0.642552,[],[],2011-12-26,0,85.0,[],Released,A newlywed couple's honeymoon is upended by th...,Newlyweds,5.9,5
4800,0,"[{""id"": 35, ""name"": ""Comedy""}, {""id"": 18, ""nam...",http://www.hallmarkchannel.com/signedsealeddel...,231617,"[{""id"": 248, ""name"": ""date""}, {""id"": 699, ""nam...",en,"Signed, Sealed, Delivered","""Signed, Sealed, Delivered"" introduces a dedic...",1.444476,"[{""name"": ""Front Street Pictures"", ""

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4803 entries, 0 to 4802
Data columns (total 20 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   budget                4803 non-null   int64  
 1   genres                4803 non-null   object 
 2   homepage              1712 non-null   object 
 3   id                    4803 non-null   int64  
 4   keywords              4803 non-null   object 
 5   original_language     4803 non-null   object 
 6   original_title        4803 non-null   object 
 7   overview              4800 non-null   object 
 8   popularity            4803 non-null   float64
 9   production_companies  4803 non-null   object 
 10  production_countries  4803 non-null   object 
 11  release_date          4802 non-null   object 
 12  revenue               4803 non-null   int64  
 13  runtime               4801 non-null   float64
 14  spoken_languages      4803 non-null   object 
 15  status               

In [4]:
data['genres'][0]

'[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]'

In [5]:
import json

In [6]:
temp = []
for i in json.loads(data['genres'][0]):
    temp.append(i.get('name', 'None'))
temp
" ".join(temp)

'Action Adventure Fantasy Science Fiction'

In [7]:
def dic_unpack(data):
    for col in ['genres', 'keywords', 'production_companies', "production_countries", "spoken_languages"]:
        data[col] = [i.get('name', 'None') for i in json.loads(data[col])]
        data[col] = " ".join(data[col])
    return data

In [8]:
data = data.apply(dic_unpack, axis=1)

In [9]:
data.columns

Index(['budget', 'genres', 'homepage', 'id', 'keywords', 'original_language',
       'original_title', 'overview', 'popularity', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'vote_average',
       'vote_count'],
      dtype='object')

In [10]:
data.head()

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,Action Adventure Fantasy Science Fiction,http://www.avatarmovie.com/,19995,culture clash future space war space colony so...,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,Ingenious Film Partners Twentieth Century Fox ...,United States of America United Kingdom,2009-12-10,2787965087,162.0,English Español,Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,Adventure Fantasy Action,http://disney.go.com/disneypictures/pirates/,285,ocean drug abuse exotic island east india trad...,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,Walt Disney Pictures Jerry Bruckheimer Films S...,United States of America,2007-05-19,961000000,169.0,English,Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500
2,245000000,Action Adventure Crime,http://www.sonypictures.com/movies/spectre/,206647,spy based on novel secret agent sequel mi6 bri...,en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,Columbia Pictures Danjaq B24,United Kingdom United States of America,2015-10-26,880674609,148.0,Français English Español Italiano Deutsch,Released,A Plan No One Escapes,Spectre,6.3,4466
3,250000000,Action Crime Drama Thriller,http://www.thedarkknightrises.com/,49026,dc comics crime fighter terrorist secret ident...,en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,Legendary Pictures Warner Bros. DC Entertainme...,United States of America,2012-07-16,1084939099,165.0,English,Released,The Legend Ends,The Dark Knight Rises,7.6,9106
4,260000000,Action Adventure Science Fiction,http://movies.disney.com/john-carter,49529,based on novel mars medallion space travel pri...,en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,Walt Disney Pictures,United States of America,2012-03-07,284139100,132.0,English,Released,"Lost in our world, found in another.",John Carter,6.1,2124


In [11]:
data = data[['original_title', 'genres', 'popularity', 'runtime', 'vote_average', 'vote_count']]
data.head()

,original_title,genres,popularity,runtime,vote_average,vote_count
0,Avatar,Action Adventure Fantasy Science Fiction,150.437577,162.0,7.2,11800
1,Pirates of the Caribbean: At World's End,Adventure Fantasy Action,139.082615,169.0,6.9,4500
2,Spectre,Action Adventure Crime,107.376788,148.0,6.3,4466
3,The Dark Knight Rises,Action Crime Drama Thriller,112.312950,165.0,7.6,9106
4,John Carter,Action Adventure Science Fiction,43.926995,132.0,6.1,2124


In [12]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4803 entries, 0 to 4802
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   original_title  4803 non-null   object 
 1   genres          4803 non-null   object 
 2   popularity      4803 non-null   float64
 3   runtime         4801 non-null   float64
 4   vote_average    4803 non-null   float64
 5   vote_count      4803 non-null   int64  
dtypes: float64(3), int64(1), object(2)
memory usage: 225.3+ KB


In [13]:
data = data.dropna()
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4801 entries, 0 to 4802
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   original_title  4801 non-null   object 
 1   genres          4801 non-null   object 
 2   popularity      4801 non-null   float64
 3   runtime         4801 non-null   float64
 4   vote_average    4801 non-null   float64
 5   vote_count      4801 non-null   int64  
dtypes: float64(3), int64(1), object(2)
memory usage: 262.6+ KB


* genres의 단어를 벡터화
* 코사인유사도를 구해 비슷한 장르의 행 검색
* 유사도가 높은 행을 기준으로 내림차순 정렬

In [14]:
from sklearn.feature_extraction.text import CountVectorizer

In [15]:
cv = CountVectorizer(ngram_range=(1, 2))
genres_mat = cv.fit_transform(data['genres'])
print(genres_mat)

  (0, 0)	1
  (0, 16)	1
  (0, 124)	1
  (0, 232)	1
  (0, 138)	1
  (0, 1)	1
  (0, 24)	1
  (0, 135)	1
  (0, 233)	1
  (1, 0)	1
  (1, 16)	1
  (1, 124)	1
  (1, 24)	1
  (1, 125)	1
  (2, 0)	1
  (2, 16)	1
  (2, 1)	1
  (2, 64)	1
  (2, 20)	1
  (3, 0)	1
  (3, 64)	1
  (3, 90)	1
  (3, 234)	1
  (3, 4)	1
  (3, 68)	1
  :	:
  (4794, 90)	1
  (4794, 234)	1
  (4794, 106)	1
  (4794, 144)	1
  (4795, 234)	1
  (4795, 153)	1
  (4795, 157)	1
  (4796, 0)	1
  (4796, 64)	1
  (4796, 234)	1
  (4796, 4)	1
  (4796, 78)	1
  (4797, 44)	1
  (4797, 214)	1
  (4797, 58)	1
  (4798, 90)	1
  (4798, 44)	1
  (4798, 214)	1
  (4798, 104)	1
  (4798, 50)	1
  (4798, 250)	1
  (4798, 182)	1
  (4798, 251)	1
  (4798, 229)	1
  (4800, 80)	1


In [16]:
genres_mat.shape

(4801, 276)

### 코사인 유사도를 사용해 genres_mat의 유사도를 산출 후 genres_sim

* 비슷하 장르의 영화를 찾기위해 계산
* 코사인 유사도는 주로 문자, 문자의 유사도를 계산하는데 사용됨

In [17]:
from sklearn.metrics.pairwise import cosine_similarity

In [18]:
genres_sim = cosine_similarity(genres_mat, genres_mat)
print(genres_sim.shape)

(4801, 4801)


In [19]:
genres_sim[:10]

array([[1.        , 0.59628479, 0.4472136 , ..., 0.        , 0.        ,
        0.        ],
       [0.59628479, 1.        , 0.4       , ..., 0.        , 0.        ,
        0.        ],
       [0.4472136 , 0.4       , 1.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.75592895, 0.3380617 , 0.50709255, ..., 0.        , 0.        ,
        0.        ],
       [0.4472136 , 0.6       , 0.2       , ..., 0.        , 0.        ,
        0.        ],
       [0.74535599, 0.8       , 0.6       , ..., 0.        , 0.        ,
        0.        ]])

In [20]:
sorted_genre_sim = genres_sim.argsort()[:,::-1]

In [21]:
sorted_genre_sim[:1]

array([[   0,   14,  870, ..., 3871, 2339, 4800]], dtype=int64)

In [22]:
data.iloc[0,:]

original_title                                      Avatar
genres            Action Adventure Fantasy Science Fiction
popularity                                      150.437577
runtime                                              162.0
vote_average                                           7.2
vote_count                                           11800
Name: 0, dtype: object

In [23]:
data.iloc[14,:]

original_title                                Man of Steel
genres            Action Adventure Fantasy Science Fiction
popularity                                       99.398009
runtime                                              143.0
vote_average                                           6.5
vote_count                                            6359
Name: 14, dtype: object

## 영화 이름과 추천받을 개수를 입력받아 출력하는 함수


In [24]:
def find_sim_movie(data, sorted_genre_sim, title_name, top_n=10):
    title_movie = data[data['original_title']== title_name]
    title_index = title_movie.index.values[0] # 원본 데이터프레임에서 영화 제목에 맞는 인덱스 추출
    similar_indexes = sorted_genre_sim[title_index, 1:(top_n+1) ] # 추출한 인덱스로 코사인유사도 결과 추출
    print("similar_index: ", similar_indexes)
    similar_indexes =similar_indexes.reshape(-1)
    return data.iloc[similar_indexes] # 추출한 코사인 유사도 결과로 원본 데이터에서 영화 추천

In [26]:
title = input("영화 제목을 정확히 입력하세요: ")
find_sim_movie(data, sorted_genre_sim, title, 10)

영화 제목을 정확히 입력하세요: The Dark Knight Rises
similar_index:  [1456 2609  405  762  761 4228 1503  629 4406 3517]


,original_title,genres,popularity,runtime,vote_average,vote_count
1456,Bound by Honor,Action Crime Drama Thriller,9.122828,180.0,7.7,115
2609,Dark Blue,Action Crime Drama Thriller,6.373316,118.0,6.5,85
405,The Fast and the Furious: Tokyo Drift,Action Crime Drama Thriller,8.282876,104.0,6.1,1705
762,Mercury Rising,Action Crime Drama Thriller,18.877795,111.0,6.0,368
761,Righteous Kill,Action Crime Drama Thriller,18.380176,101.0,5.9,375
4230,Killing Zoe,Action Crime Drama Thriller,5.817519,96.0,6.1,111
1503,Takers,Action Crime Drama Thriller,18.472420,107.0,6.0,394
629,Need for Speed,Action Crime Drama Thriller,54.814890,130.0,6.1,1520
4408,Jimmy and Judy,Action Crime Drama Thriller,1.255771,99.0,5.4,8
3518,ต้มยำกุ้ง,Action Crime Drama Thriller,11.818320,108.0,6.8,167
